In [ ]:
import json
import logging

import joblib
import numpy as np
import pandas as pd

import shared

# time poetry run python static_individual_estimates.py

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

import static_individual_estimates

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={"figure.figsize":(16, 9)}) 

In [ ]:
runs = pd.read_csv(f'data/runs_{shared.race_id_str()}.tsv', delimiter='\t')
runs = runs.query("num_runs > 2")
#runs["log_pace"] = np.log(runs["pace"])
runs

In [ ]:
by_name = pd.DataFrame(data=runs.groupby("name")["pace"].apply(list).items(),
                       columns=["name", "paces"])
by_name

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import Normalizer

In [ ]:
def calculate_coef(row):
    name = row["name"]
    paces = row["paces"]
    X = np.array(paces).reshape(-1,1) 
    #y = np.array(row["personal_coefficients"]).reshape(len(terrain_coefficients),1) 
    #cov = EllipticEnvelope(random_state=2021, contamination=0.001).fit(X)
    #predictions = cov.predict(X)
    normalized = X / np.mean(X)
    #predictions = LocalOutlierFactor(contamination=0.05).fit_predict(X)
    predictions = normalized > 1.2
    # imputed = paces
    #    imputed[predictions] = np.mean(X)
    logging.info(f"{name} normalized: {normalized}, predictions: {predictions}, paces: {X}")
    return [predictions]

by_name[["predictions"]] = by_name.apply(calculate_coef, axis=1, result_type="expand")  
by_name.sample(10).round(4)



In [ ]:
"nan".split(",")

In [ ]:
runner_means = runs[["name", "pace"]].groupby(["name"]).agg("mean")
runs["pace_mean"] = runner_means["pace"][runs["name"]].values
runs["outlier"] = (runs["pace"] / runs["pace_mean"]) > 1.25
runs["fixed_pace"] = runs["pace"]
runs.loc[runs["outlier"]==True, "fixed_pace"] = runs[runs["outlier"]==True]["pace_mean"]
runs

In [ ]:
runs["outlier"].mean()